In [2]:
from flask import Flask, jsonify
import oracledb
import numpy as np 
import os
import json
from datetime import datetime
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import base64
import locale
from fpdf import FPDF

In [3]:
# Conexão ao banco de dados

# Criando o DataFrame com os dados e os nomes das colunas 
df = pd.DataFrame(rows, columns=columns)

In [5]:
print(df.columns)

Index(['OEE_APONTAMENTO', 'FAB_PROGRAMADO_OP', 'OP', 'CODGER2', 'PRODUTO',
       'DATAENTRADA', 'DATA_INICIO_OPERACAO', 'DATASAIDA', 'QUANT_PROG',
       'QUANT_APONTADA', 'QUANT_PERDA', 'QUANT_APONTADA_SALDO',
       'COD_UNIDADE_OPERATIVA', 'UNIDADE_OPERATIVA', 'REPETIR_APONTAMENTO',
       'FATOR_DIVISAO', 'REVOGADO', 'ENVIO_REBATIDA', 'CARRINHO_PRODUCAO',
       'OEE_CELULA_MANUFATURA', 'CODGER', 'PERDA', 'NOME_FAMILIA',
       'UNIDADE_OPERATIVA_DESTINO_POSSIVEL',
       'UNIDADE_OPERATIVA_DESTINO_EFETIVA'],
      dtype='object')


In [6]:
df = df.drop(['UNIDADE_OPERATIVA_DESTINO_POSSIVEL','UNIDADE_OPERATIVA_DESTINO_EFETIVA',
              'FATOR_DIVISAO', 'REPETIR_APONTAMENTO','REVOGADO', 'ENVIO_REBATIDA','OEE_CELULA_MANUFATURA','CARRINHO_PRODUCAO'], axis=1)

In [7]:
df

,OEE_APONTAMENTO,FAB_PROGRAMADO_OP,OP,CODGER2,PRODUTO,DATAENTRADA,DATA_INICIO_OPERACAO,DATASAIDA,QUANT_PROG,QUANT_APONTADA,QUANT_PERDA,QUANT_APONTADA_SALDO,COD_UNIDADE_OPERATIVA,UNIDADE_OPERATIVA,CODGER,PERDA,NOME_FAMILIA
0,250304,1876352,103102,01.12.1084,VW960.02P,2024-08-14 16:36:21,NaT,2024-08-14 16:36:21,57.0,5.0,0.0,52.0,42,CORTE / PLASMA / MAQ-281,01.12.1084,0.0,PARABOLICA
1,250328,1876321,103102,01.12.1081,VW940.01P,2024-08-14 16:47:15,NaT,2024-08-14 16:47:15,23.0,1.0,0.0,22.0,42,CORTE / PLASMA / MAQ-281,01.12.1081,0.0,PARABOLICA
2,250331,1876321,103102,01.12.1081,VW940.01P,2024-08-14 16:48:42,NaT,2024-08-14 16:48:42,23.0,1.0,0.0,22.0,42,CORTE / PLASMA / MAQ-281,01.12.1081,0.0,PARABOLICA
3,250362,1876342,103102,01.12.1083,VW960.01P,2024-08-14 17:01:52,NaT,2024-08-14 17:01:52,12.0,1.0,0.0,11.0,42,CORTE / PLASMA / MAQ-281,01.12.1083,0.0,PARABOLICA
4,250374,1867912,102862,01.02.7801,NM2646.01,2024-08-14 23:05:28,NaT,2024-08-14 23:05:28,198.0,30.0,0.0,168.0,38,CORTE / CISALHAMENTO / MAQ-319,01.02.7801,0.0,MOLAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429210,201989,1854088,102454,01.02.7275,RD851.01,2024-06-28 19:22:21,NaT,2024-06-28 19:22:21,395.0,5.0,0.0,390.0,39,CORTE / CISALHAMENTO / MAQ-409,01.02.7275,0.0,MOLAS
429211,175106,1821752,101492,03.01.7286,14.92.D.9736,2024-04-25 10:39:52,2024-11-13,2024-04-25 10:39:52,735.0,65.0,0.0,670.0,20,TRATAMENTO DE SUPERFICIE / INTERNO / KTL-001,03.01.7286,0.0,GRAMPOS
429212,116988,1763031,100029,01.02.1032,MB45.05,2023-11-10 13:01:45,NaT,2023-11-10 13:01:45,20.0,20.0,0.0,0.0,40,CORTE / CISALHAMENTO / MAQ-336,01.02.1032,0.0,MOLAS
429213,238893,1870111,102937,01.02.8799,IV60.02,2024-08-06 09:09:50,NaT,2024-08-06 09:09:50,53.0,1.0,0.0,52.0,39,CORTE / CISALHAMENTO / MAQ-409,01.02.8799,0.0,MOLAS


In [8]:
# # Filtrar os dados para manter apenas a OP 104682
# df = df[df['OP'] == 104682]

In [9]:
# Valores únicos na coluna 'NOME_FAMILIA'
valores_unicos = df['NOME_FAMILIA'].unique()

# Exibindo os valores únicos
print(valores_unicos)

['PARABOLICA' 'MOLAS' 'GRAMPOS' 'AGULHAS' 'ESPIGOES'
 'EXPANDER/CONJ EXPANDER' 'PINOS' 'BUCHAS OLHETE']


In [10]:
# Filtra as famílias 

df1 = df[df['NOME_FAMILIA'].isin(['AGULHAS', 'GRAMPOS'])].copy()


In [11]:
print(df1.columns)

Index(['OEE_APONTAMENTO', 'FAB_PROGRAMADO_OP', 'OP', 'CODGER2', 'PRODUTO',
       'DATAENTRADA', 'DATA_INICIO_OPERACAO', 'DATASAIDA', 'QUANT_PROG',
       'QUANT_APONTADA', 'QUANT_PERDA', 'QUANT_APONTADA_SALDO',
       'COD_UNIDADE_OPERATIVA', 'UNIDADE_OPERATIVA', 'CODGER', 'PERDA',
       'NOME_FAMILIA'],
      dtype='object')


In [12]:
df1.tail(5)

,OEE_APONTAMENTO,FAB_PROGRAMADO_OP,OP,CODGER2,PRODUTO,DATAENTRADA,DATA_INICIO_OPERACAO,DATASAIDA,QUANT_PROG,QUANT_APONTADA,QUANT_PERDA,QUANT_APONTADA_SALDO,COD_UNIDADE_OPERATIVA,UNIDADE_OPERATIVA,CODGER,PERDA,NOME_FAMILIA
429205,19946,1624939,95539,03.01.0073,018.82.D.73,2023-02-06 10:08:04,2024-10-02,2023-02-06 10:08:04,10.0,10.0,0.0,0.0,32,EMBALAGEM / EMBALAGEM 1 / EMB-001,03.01.0073,0.0,GRAMPOS
429206,60017,1700720,98494,03.01.7187,024.130.C.9580,2023-07-11 12:59:02,2024-10-02,2023-07-11 12:59:02,315.0,50.0,0.0,265.0,32,EMBALAGEM / EMBALAGEM 1 / EMB-001,03.01.7187,0.0,GRAMPOS
429207,187029,1838417,101991,03.01.0609,14.77.B.606,2024-06-05 13:28:32,2024-11-13,2024-06-05 13:28:32,5.0,5.0,0.0,0.0,20,TRATAMENTO DE SUPERFICIE / INTERNO / KTL-001,03.01.0609,0.0,GRAMPOS
429211,175106,1821752,101492,03.01.7286,14.92.D.9736,2024-04-25 10:39:52,2024-11-13,2024-04-25 10:39:52,735.0,65.0,0.0,670.0,20,TRATAMENTO DE SUPERFICIE / INTERNO / KTL-001,03.01.7286,0.0,GRAMPOS
429214,224607,1858173,102560,03.01.0096,018.82.D.96,2024-07-23 08:44:08,2024-10-02,2024-07-23 08:44:08,450.0,11.0,0.0,439.0,32,EMBALAGEM / EMBALAGEM 1 / EMB-001,03.01.0096,0.0,GRAMPOS


In [13]:
# Converte "DATAENTRADA" para datetime se necessário
df1['DATAENTRADA'] = pd.to_datetime(df1['DATAENTRADA'])
df1['DATASAIDA'] = pd.to_datetime(df1['DATASAIDA'])

# Cria uma nova coluna "Hora entrada" e "Data saida"
df1['Hora Entrada'] = df1['DATAENTRADA'].dt.time
df1['Data Entrada'] = df1['DATAENTRADA'].dt.date

df1['Hora Saida'] = df1['DATASAIDA'].dt.time
df1['Data Saida'] = df1['DATASAIDA'].dt.date

In [14]:
df1.tail(5)

,OEE_APONTAMENTO,FAB_PROGRAMADO_OP,OP,CODGER2,PRODUTO,DATAENTRADA,DATA_INICIO_OPERACAO,DATASAIDA,QUANT_PROG,QUANT_APONTADA,...,QUANT_APONTADA_SALDO,COD_UNIDADE_OPERATIVA,UNIDADE_OPERATIVA,CODGER,PERDA,NOME_FAMILIA,Hora Entrada,Data Entrada,Hora Saida,Data Saida
429205,19946,1624939,95539,03.01.0073,018.82.D.73,2023-02-06 10:08:04,2024-10-02,2023-02-06 10:08:04,10.0,10.0,...,0.0,32,EMBALAGEM / EMBALAGEM 1 / EMB-001,03.01.0073,0.0,GRAMPOS,10:08:04,2023-02-06,10:08:04,2023-02-06
429206,60017,1700720,98494,03.01.7187,024.130.C.9580,2023-07-11 12:59:02,2024-10-02,2023-07-11 12:59:02,315.0,50.0,...,265.0,32,EMBALAGEM / EMBALAGEM 1 / EMB-001,03.01.7187,0.0,GRAMPOS,12:59:02,2023-07-11,12:59:02,2023-07-11
429207,187029,1838417,101991,03.01.0609,14.77.B.606,2024-06-05 13:28:32,2024-11-13,2024-06-05 13:28:32,5.0,5.0,...,0.0,20,TRATAMENTO DE SUPERFICIE / INTERNO / KTL-001,03.01.0609,0.0,GRAMPOS,13:28:32,2024-06-05,13:28:32,2024-06-05
429211,175106,1821752,101492,03.01.7286,14.92.D.9736,2024-04-25 10:39:52,2024-11-13,2024-04-25 10:39:52,735.0,65.0,...,670.0,20,TRATAMENTO DE SUPERFICIE / INTERNO / KTL-001,03.01.7286,0.0,GRAMPOS,10:39:52,2024-04-25,10:39:52,2024-04-25
429214,224607,1858173,102560,03.01.0096,018.82.D.96,2024-07-23 08:44:08,2024-10-02,2024-07-23 08:44:08,450.0,11.0,...,439.0,32,EMBALAGEM / EMBALAGEM 1 / EMB-001,03.01.0096,0.0,GRAMPOS,08:44:08,2024-07-23,08:44:08,2024-07-23


In [15]:
# Renomeando colunas específicas
df1 = df1.rename(columns={
    'COD_UNIDADE_OPERATIVA': 'Cd.Unid.Op',
    'FAB_PROGRAMADO_OP': 'Etiqueta', 
    'DATASAIDA': 'Data Saida',
    'DATAENTRADA': 'Data Entrada',
    'QUANT_APONTADA': 'Produzido',
    'QUANT_PERDA': 'Perda',
    'DATAENTRADA': 'Data_entrada+hora',
    'DATASAIDA': 'Data_saida+hora'
    
})

In [16]:
# Definir o mapeamento de 'Cd.Unid.Op' para 'Unidade'
unidade_mapping = {
    57: "BASTIDOR",
    1: "APONTAMENTO",
    5: "APONTAMENTO",
    16: "APONTAMENTO",
    17: "APONTAMENTO",
    25: "APONTAMENTO",
    3: "CORTE",
    15: "CORTE",
    64: "CORTE",
    71: "CONFERENCIA",
    10: "DOBRA FRIA",
    11: "DOBRA FRIA",
    12: "DOBRA FRIA",
    37: "DOBRA QUENTE",
    43: "DOBRA QUENTE",
    9: "DOBRA INTERMEDIARIA",
    66: "DOBRA QUENTE",
    67: "DOBRA QUENTE",
    32: "EMBALAGEM",
    2: "GRAVAÇÃO",
    4: "GRAVAÇÃO",
    19: "GRAVAÇÃO",
    6: "LAMINAÇÃO",
    7: "LAMINAÇÃO",
    8: "LAMINAÇÃO",
    13: "LAMINAÇÃO",
    14: "LAMINAÇÃO",
    65: "LAM. PÓS TRAT. TÉRMICO",
    21: "MONTAGEM ACESSORIOS",
    49: "AJUSTAGEM",
    63: "TORNO",
    59: "SAÍDA ACAB. EXTERNO NF",
    60: "RECEB. ACAB. EXTERNO",
    20: "KTL",
    61: "OUTROS ACAB. SUP.",
    53: "TRAT. TÉRMICO",
    54: "TRAT. TÉRMICO",
    52: "ACAB. EXTERNO"
}

# Adicionar a nova coluna 'Unidade' ao DataFrame 'df1' com base no mapeamento
df1['Unidade'] = df1['Cd.Unid.Op'].map(unidade_mapping)

# # Exibir as primeiras linhas do DataFrame atualizado para ver a nova coluna 'Unidade'
# import ace_tools as tools; tools.display_dataframe_to_user("DataFrame Atualizado com Unidade", df1)

# Garantir que as colunas de data estão no formato correto
df1['Data Saida'] = pd.to_datetime(df1['Data Saida'], format='%d/%m/%Y')

# Agrupar os dados por 'Data Saida' e 'Unidade', e somar a coluna 'Produzido'
df_pivot = df1.groupby(['Data Saida', 'Unidade'])['Produzido'].sum().reset_index()

# Criar uma tabela dinâmica (pivot) para organizar os dados como no exemplo
df_pivot = df_pivot.pivot(index='Data Saida', columns='Unidade', values='Produzido')

# Preencher valores NaN com 0 (caso necessário)
df_pivot = df_pivot.fillna(0)



In [17]:
df_pivot

Unidade,ACAB. EXTERNO,AJUSTAGEM,APONTAMENTO,BASTIDOR,CORTE,DOBRA FRIA,DOBRA INTERMEDIARIA,DOBRA QUENTE,EMBALAGEM,GRAVAÇÃO,KTL,LAM. PÓS TRAT. TÉRMICO,LAMINAÇÃO,MONTAGEM ACESSORIOS,OUTROS ACAB. SUP.,SAÍDA ACAB. EXTERNO NF,TORNO,TRAT. TÉRMICO
Data Saida,,,,,,,,,,,,,,,,,,
2022-10-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1583541.0,0.0,0.0,0.0,0.0,0.0
2022-10-07,0.0,0.0,0.0,0.0,0.0,2329.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-13,0.0,0.0,0.0,0.0,0.0,1402.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-14,0.0,0.0,0.0,0.0,0.0,1990.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-18,0.0,0.0,0.0,0.0,320.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-14,0.0,1589.0,5045.0,0.0,5473.0,2760.0,0.0,1737.0,2510.0,6456.0,2680.0,0.0,4013.0,0.0,0.0,0.0,0.0,3780.0
2025-03-15,0.0,0.0,1656.0,0.0,1870.0,0.0,0.0,1521.0,0.0,1615.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,572.0
2025-03-17,0.0,2117.0,3061.0,175.0,4835.0,3385.0,0.0,2277.0,2090.0,3256.0,1250.0,0.0,5611.0,0.0,0.0,0.0,0.0,3434.0


In [18]:
# tabela dinamica com produção total do dia 

# Garantir que as colunas de data estão no formato correto
df1['Data Saida'] = pd.to_datetime(df1['Data Saida'], format='%d/%m/%Y')

# Filtrar as linhas onde 'Produzido' e 'Perda' são diferentes
df_filtrado = df1[df1['Produzido'] != df1['Perda']]

# Agrupar os dados por 'Data Saida' e 'Unidade', e somar a coluna 'Produzido'
df_pivot = df_filtrado.groupby(['Data Saida', 'Unidade'])['Produzido'].sum().reset_index()

# Criar uma tabela dinâmica (pivot) para organizar os dados
df_pivot = df_pivot.pivot(index='Data Saida', columns='Unidade', values='Produzido')

# Preencher valores NaN com 0 (caso necessário)
df_pivot_produção = df_pivot.fillna(0)

In [19]:
df_pivot_produção

Unidade,ACAB. EXTERNO,AJUSTAGEM,APONTAMENTO,BASTIDOR,CORTE,DOBRA FRIA,DOBRA INTERMEDIARIA,DOBRA QUENTE,EMBALAGEM,GRAVAÇÃO,KTL,LAM. PÓS TRAT. TÉRMICO,LAMINAÇÃO,MONTAGEM ACESSORIOS,OUTROS ACAB. SUP.,SAÍDA ACAB. EXTERNO NF,TORNO,TRAT. TÉRMICO
Data Saida,,,,,,,,,,,,,,,,,,
2022-10-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1583541.0,0.0,0.0,0.0,0.0,0.0
2022-10-07,0.0,0.0,0.0,0.0,0.0,2329.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-13,0.0,0.0,0.0,0.0,0.0,1402.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-14,0.0,0.0,0.0,0.0,0.0,1990.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-18,0.0,0.0,0.0,0.0,320.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-14,0.0,1589.0,5045.0,0.0,5473.0,2760.0,0.0,1737.0,2425.0,6456.0,2680.0,0.0,4013.0,0.0,0.0,0.0,0.0,3744.0
2025-03-15,0.0,0.0,1656.0,0.0,1870.0,0.0,0.0,1521.0,0.0,1615.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,572.0
2025-03-17,0.0,2117.0,3061.0,175.0,4835.0,3385.0,0.0,2277.0,2090.0,3256.0,1250.0,0.0,5611.0,0.0,0.0,0.0,0.0,3434.0


In [20]:
# Garantir que 'Data Entrada' e 'Hora Entrada' estejam no formato correto
df1['Data Entrada'] = pd.to_datetime(df1['Data Entrada'], format='%d/%m/%Y', errors='coerce')
df1['Hora Entrada'] = pd.to_datetime(df1['Hora Entrada'], format='%H:%M:%S', errors='coerce').dt.time

# Criar a coluna unificada 'Data_entrada+hora' combinando 'Data Entrada' e 'Hora Entrada'
df1['Data_entrada+hora'] = pd.to_datetime(df1['Data Entrada'].astype(str) + ' ' + df1['Hora Entrada'].astype(str))

# Verificar se existem duplicatas no DataFrame original com base em 'Data_entrada+hora', 'Cd.Unid.Op', e 'Etiqueta'
duplicatas = df1[df1.duplicated(subset=['Data_entrada+hora', 'Cd.Unid.Op', 'Etiqueta'], keep=False)]


# Remover as duplicatas com base nas colunas 'Data_entrada+hora', 'Cd.Unid.Op', e 'Etiqueta'
df1_unique = df1.drop_duplicates(subset=['Data_entrada+hora', 'Cd.Unid.Op', 'Etiqueta']).copy()

# Usar .loc[] para modificar o DataFrame sem gerar o warning
df1_unique.loc[:, 'Data'] = df1_unique['Data_entrada+hora'].dt.date

# Certificar-se de que não há mais duplicatas antes de continuar
#print("Verificando duplicatas após remoção:")
duplicatas_restantes = df1_unique[df1_unique.duplicated(subset=['Data_entrada+hora', 'Cd.Unid.Op', 'Etiqueta'], keep=False)]
#print(duplicatas_restantes)

# Agrupa por 'Cd.Unid.Op' e 'Data' e calcula o menor e o maior horário de entrada por unidade, sem duplicatas
min_max_time = df1_unique.groupby(['Cd.Unid.Op', 'Data']).agg(
    menor_horario=('Data_entrada+hora', 'min'),
    maior_horario=('Data_saida+hora', 'max')
).reset_index()

# Calcula o tempo de produção como a diferença entre o maior e menor horário
min_max_time['Tempo_programado'] = (min_max_time['maior_horario'] - min_max_time['menor_horario']).dt.total_seconds()


# Mapear os códigos de unidade para os nomes de unidade
min_max_time['Unidade'] = min_max_time['Cd.Unid.Op'].map(unidade_mapping)

# Agrupar e somar o 'Tempo_programado' para garantir que não haja duplicatas
min_max_time_agg = min_max_time.groupby(['Data', 'Unidade'])['Tempo_programado'].sum().reset_index()

# Criar a tabela dinâmica
df_tempo_programado = min_max_time_agg.pivot(index='Data', columns='Unidade', values='Tempo_programado')

# Preencher valores NaN com 0 (caso necessário)
df_tempo_programado = df_tempo_programado.fillna(0)

# Garantir que 'Data Entrada' e 'Hora Entrada' estejam no formato correto
df1['Data Entrada'] = pd.to_datetime(df1['Data Entrada'], format='%d/%m/%Y', errors='coerce')
df1['Hora Entrada'] = pd.to_datetime(df1['Hora Entrada'], format='%H:%M:%S', errors='coerce')
df1['Hora Saida'] = pd.to_datetime(df1['Hora Saida'], format='%H:%M:%S', errors='coerce')

# Criar a coluna unificada 'Data_entrada+hora' combinando 'Data Entrada' e 'Hora Entrada'
df1['Data_entrada+hora'] = pd.to_datetime(df1['Data Entrada'].astype(str) + ' ' + df1['Hora Entrada'].astype(str))

# Remover duplicatas com base em 'Data_entrada+hora', 'Cd.Unid.Op', e 'Etiqueta' antes de calcular as horas produzidas
df1_unique_produzidas = df1.drop_duplicates(subset=['Data_entrada+hora', 'Cd.Unid.Op', 'Etiqueta']).copy()

# Calcular a diferença de horas entre "Hora Entrada" e "Hora Saida" para o DataFrame sem duplicatas
df1_unique_produzidas['Horas Produzidas'] = (df1_unique_produzidas['Hora Saida'] - df1_unique_produzidas['Hora Entrada']).dt.total_seconds()

# Agrupar os dados por 'Data Saida' e 'Unidade', somando as horas produzidas sem duplicatas
df_horas_pivot = df1_unique_produzidas.groupby(['Data Saida', 'Unidade'])['Horas Produzidas'].sum().reset_index()

# Converter a coluna 'Data Saida' para o formato de data
df_horas_pivot['Data Saida'] = pd.to_datetime(df_horas_pivot['Data Saida'], format='%d/%m/%Y', errors='coerce')

# Criar uma tabela dinâmica para organizar os dados por dia e setor
df_horas_pivot = df_horas_pivot.pivot(index='Data Saida', columns='Unidade', values='Horas Produzidas')

# Preencher valores NaN com 0
df_tempo_produzido = df_horas_pivot.fillna(0)



C:\Users\HP\AppData\Local\Temp\ipykernel_15800\1528609992.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['Data_entrada+hora'] = pd.to_datetime(df1['Data Entrada'].astype(str) + ' ' + df1['Hora Entrada'].astype(str))


In [21]:
df_tempo_produzido

Unidade,ACAB. EXTERNO,AJUSTAGEM,APONTAMENTO,BASTIDOR,CORTE,DOBRA FRIA,DOBRA INTERMEDIARIA,DOBRA QUENTE,EMBALAGEM,GRAVAÇÃO,KTL,LAM. PÓS TRAT. TÉRMICO,LAMINAÇÃO,MONTAGEM ACESSORIOS,OUTROS ACAB. SUP.,SAÍDA ACAB. EXTERNO NF,TORNO,TRAT. TÉRMICO
Data Saida,,,,,,,,,,,,,,,,,,
2022-10-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6366.0,0.0,0.0,0.0,0.0,0.0
2022-10-07,0.0,0.0,0.0,0.0,0.0,14933.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-13,0.0,0.0,0.0,0.0,0.0,19457.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-14,0.0,0.0,0.0,0.0,0.0,18647.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-18,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-14,0.0,16.0,31177.0,0.0,0.0,781.0,0.0,37797.0,24.0,39.0,6.0,0.0,33322.0,0.0,0.0,0.0,0.0,38.0
2025-03-15,0.0,0.0,1896.0,0.0,0.0,0.0,0.0,21.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2025-03-17,0.0,19.0,27796.0,2.0,0.0,33.0,0.0,18874.0,48.0,70.0,2.0,0.0,33789.0,0.0,0.0,0.0,0.0,40.0


In [59]:
#nova linha de código para analise de ajuste em horas produzidas
#SE TEMPO PRODUZIDO < 5 SEGUNDOS
#USAR O CICLO TEÓRICO x QUANTIDADE PRODUZIDA
#definir ciclo de cada setor (SEGUNDOS)

ciclos_teoricos= {
    'CORTE': 6.11,
    'GRAVAÇÃO': 7.5,
    'APONTAMENTO': 9.00,
    'LAMINAÇÃO': 11.05 ,
    'DOBRA FRIA': 8.00 ,
    'DOBRA QUENTE': 12 ,
    'TRAT. TÉRMICO': 18 ,
    'AJUSTAGEM': 6 ,
    'ACAB. EXTERNO': 9 ,
    'KTL': 7.2 ,
    'EMBALAGEM': 7.2,
    'TORNO': 55.38
}

# Converter o dicionário de ciclos teóricos em um DataFrame alinhado ao df_pivot_produção
df_ciclo_teorico = pd.DataFrame([ciclos_teoricos], index=df_tempo_produzido.index).reindex(columns=df_tempo_produzido.columns)

# Aplicar a lógica de substituição
df_tempo_produzido_corrigido = df_tempo_produzido.copy()

# Substituir os valores menores que 4 segundos
df_tempo_produzido_corrigido = df_tempo_produzido_corrigido.applymap(
    lambda x: x if x >= 15 else np.nan
)

# Preencher os valores NaN com df_pivot_produção * ciclo_teorico
df_tempo_produzido_corrigido.fillna(df_pivot_produção * df_ciclo_teorico, inplace=True)

# Visualizar a saída corrigida
(df_tempo_produzido_corrigido)



C:\Users\HP\AppData\Local\Temp\ipykernel_15800\1423139288.py:28: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



Unidade,ACAB. EXTERNO,AJUSTAGEM,APONTAMENTO,BASTIDOR,CORTE,DOBRA FRIA,DOBRA INTERMEDIARIA,DOBRA QUENTE,EMBALAGEM,GRAVAÇÃO,KTL,LAM. PÓS TRAT. TÉRMICO,LAMINAÇÃO,MONTAGEM ACESSORIOS,OUTROS ACAB. SUP.,SAÍDA ACAB. EXTERNO NF,TORNO,TRAT. TÉRMICO
Data Saida,,,,,,,,,,,,,,,,,,
2022-10-06,0.0,0.0,0.0,NaN,0.00,0.0,NaN,0.0,0.0,0.0,0.0,NaN,6366.0,NaN,NaN,NaN,0.0,0.0
2022-10-07,0.0,0.0,0.0,NaN,0.00,14933.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0
2022-10-13,0.0,0.0,0.0,NaN,0.00,19457.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0
2022-10-14,0.0,0.0,0.0,NaN,0.00,18647.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0
2022-10-18,0.0,0.0,0.0,NaN,23.00,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-14,0.0,16.0,31177.0,NaN,33440.03,781.0,NaN,37797.0,24.0,39.0,19296.0,NaN,33322.0,NaN,NaN,NaN,0.0,38.0
2025-03-15,0.0,0.0,1896.0,NaN,11425.70,0.0,NaN,21.0,0.0,12112.5,0.0,NaN,0.0,NaN,NaN,NaN,0.0,10296.0
2025-03-17,0.0,19.0,27796.0,NaN,29541.85,33.0,NaN,18874.0,48.0,70.0,9000.0,NaN,33789.0,NaN,NaN,NaN,0.0,40.0


In [43]:
#DISPONIBILIDADE

# Garantir que ambos os DataFrames estão com o mesmo índice e colunas
df_tempo_produzido_corrigido = df_tempo_produzido_corrigido.reindex_like(df_tempo_programado)

# Dividir diretamente os valores de 'Horas Produzidas' pelo 'Tempo Programado'
df_disponibilidade = (df_tempo_produzido_corrigido / df_tempo_programado) * 100 #coloca em % 
df_disponibilidade = df_disponibilidade.round(2) #deixa apenas 2 casas decimais

# Adicionar a coluna 'Disponibilidade' diretamente no DataFrame resultante
df_disponibilidade.columns.name = 'Unidade'
df_disponibilidade.index.name = 'Data'

# Preencher valores NaN com 0
df_disponibilidade = df_disponibilidade.fillna(0)
df_disponibilidade

Unidade,ACAB. EXTERNO,AJUSTAGEM,APONTAMENTO,BASTIDOR,CORTE,DOBRA FRIA,DOBRA INTERMEDIARIA,DOBRA QUENTE,EMBALAGEM,GRAVAÇÃO,KTL,LAM. PÓS TRAT. TÉRMICO,LAMINAÇÃO,MONTAGEM ACESSORIOS,OUTROS ACAB. SUP.,SAÍDA ACAB. EXTERNO NF,TORNO,TRAT. TÉRMICO
Data,,,,,,,,,,,,,,,,,,
2022-10-06,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,28.64,0.0,0.0,0.0,0.0,0.00
2022-10-07,0.0,0.00,0.00,0.00,0.00,46.98,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00
2022-10-13,0.0,0.00,0.00,0.00,0.00,79.70,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00
2022-10-14,0.0,0.00,0.00,0.00,0.00,62.65,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00
2022-10-18,0.0,0.00,0.00,0.00,0.58,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-14,0.0,0.12,48.23,0.00,106.96,1.32,0.0,27.67,0.07,0.07,0.02,0.0,58.37,0.0,0.0,0.0,0.0,0.03
2025-03-15,0.0,0.00,15.08,0.00,48.23,0.00,0.0,0.05,0.00,0.06,0.00,0.0,0.00,0.0,0.0,0.0,0.0,2434.04
2025-03-17,0.0,0.07,80.80,0.00,52.07,0.09,0.0,21.02,0.15,0.13,45.50,0.0,34.87,0.0,0.0,0.0,0.0,0.04


In [44]:
#PERFOMANCE

#definir ciclo de cada setor (SEGUNDOS)
ciclos_teoricos= {
    'CORTE': 6.11,
    'GRAVAÇÃO': 7.5,
    'APONTAMENTO': 9.00,
    'LAMINAÇÃO': 11.05 ,
    'DOBRA FRIA': 8.00 ,
    'DOBRA QUENTE': 12 ,
    'TRAT. TÉRMICO': 18 ,
    'AJUSTAGEM': 6 ,
    'ACAB. EXTERNO': 9 ,
    'KTL': 7.2 ,
    'EMBALAGEM': 7.2,
    'TORNO': 55.38
}

# Criar uma nova tabela para armazenar a performance
df_performance = df_tempo_produzido.copy()  # Usar a tabela de horas produzidas

# Calcular a performance para cada setor
for setor in df_performance.columns:
    ciclo_teorico = ciclos_teoricos.get(setor, 1)  # Ciclo teórico do setor
    # Calcular performance = (produção * ciclo teórico) / horas produzidas
    df_performance[setor] = (((df_pivot_produção[setor] * ciclo_teorico) / (df_tempo_produzido_corrigido[setor])))*100  # Converter horas para segundos
    df_performance[setor] = df_performance[setor].round(2)  # Arredondar para 2 casas decimais
    

# Exibir o resultado da performance
(df_performance.head())

df_performance=df_performance.fillna(0)


In [45]:
#Qualidade

#nova tabela dinamica com o saldo (produção-perda) 
# Garantir que as colunas necessárias estão presentes
# Criar uma nova coluna 'Saldo' com a fórmula: Saldo = Produzido - Perda

df1['Saldo'] = df1['Produzido'] - df1['Perda']

# Gerar a tabela dinâmica com base no 'Saldo' por dia e setor
df_saldo_pivot = df1.groupby(['Data Saida', 'Unidade'])['Saldo'].sum().reset_index()

# Converter a coluna 'Data Saida' para o formato de data correto
df_saldo_pivot['Data Saida'] = pd.to_datetime(df_saldo_pivot['Data Saida'], format='%d/%m/%Y', errors='coerce')

# Criar uma tabela dinâmica para organizar os dados de saldo por dia e setor
df_saldo_pivot = df_saldo_pivot.pivot(index='Data Saida', columns='Unidade', values='Saldo')

# Preencher valores NaN com 0 (caso necessário)
df_saldo_pivot = df_saldo_pivot.fillna(0)

# Criar uma nova tabela para armazenar a qualidade
df_qualidade = df_saldo_pivot.copy()

# Calcular a qualidade para cada setor
for setor in df_qualidade.columns:
    # Qualidade = Saldo / Produção Total (da tabela dinâmica de produção)
    df_qualidade[setor] = (df_saldo_pivot[setor] / df_pivot[setor]) * 100
    df_qualidade[setor] = df_qualidade[setor].round(2)  # Arredondar para 2 casas decimais


# Preencher valores NaN com 0 (caso necessário)
df_qualidade = df_qualidade.fillna(0)

# # Exibir o resultado da qualidade
# (df_qualidade.())

In [46]:
#OEE

#nova tabela dinamica com o oee por dia de cada setor

# Criar uma nova tabela para armazenar o OEE
df_oee = pd.DataFrame(index=df_disponibilidade.index, columns=df_disponibilidade.columns)

# Calcular o OEE para cada setor
for setor in df_oee.columns:
    # OEE = Disponibilidade * Performance * Qualidade
    df_oee[setor] = ((df_disponibilidade[setor] * df_performance[setor] * df_qualidade[setor])) / 10000
    df_oee[setor] = df_oee[setor].round(2)  # Arredondar para 2 casas decimais

#nan com 0 

df_oee = df_oee.fillna(0)

(df_oee)

Unidade,ACAB. EXTERNO,AJUSTAGEM,APONTAMENTO,BASTIDOR,CORTE,DOBRA FRIA,DOBRA INTERMEDIARIA,DOBRA QUENTE,EMBALAGEM,GRAVAÇÃO,KTL,LAM. PÓS TRAT. TÉRMICO,LAMINAÇÃO,MONTAGEM ACESSORIOS,OUTROS ACAB. SUP.,SAÍDA ACAB. EXTERNO NF,TORNO,TRAT. TÉRMICO
Data,,,,,,,,,,,,,,,,,,
2022-10-06,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00
2022-10-07,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00
2022-10-13,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00
2022-10-14,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00
2022-10-18,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-14,0.0,71.50,70.24,0.00,106.96,37.32,0.0,14.54,50.93,86.91,64.32,0.0,77.68,0.0,0.0,0.0,0.0,53.20
2025-03-15,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00
2025-03-17,0.0,46.80,80.08,0.00,52.07,73.85,0.0,30.43,47.02,45.35,45.50,0.0,63.99,0.0,0.0,0.0,0.0,61.81


In [47]:
# Supondo que df_oee já foi criado corretamente com as datas como índice e as unidades como colunas
df_oee.index = pd.to_datetime(df_oee.index, format='%d/%m/%Y')

# Filtrar para mostrar apenas os dados de 2024 e até o dia anterior ao atual
data_atual = datetime.now().date()
df_oee = df_oee[(df_oee.index.year == 2025) & (df_oee.index < pd.Timestamp(data_atual))]
df_oee = df_oee[~df_oee.index.weekday.isin([5, 6])]

# Extrair os meses do DataFrame para criar os filtros
df_oee['Month'] = df_oee.index.strftime('%B')  # Adiciona uma coluna com o nome do mês
months = df_oee['Month'].unique()

# Criar o gráfico de linha com marcadores para a coluna 'DOBRA QUENTE'
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.05)

# Adicionar a linha com os pontos de produção para 'DOBRA QUENTE' (inicialmente mostrando todos os meses de 2024)
fig.add_trace(go.Scatter(
    x=df_oee.index, 
    y=df_oee['DOBRA QUENTE'],
    mode='lines+markers',
    name='OEE',  # Nome da linha na legenda
    marker=dict(size=8),
    line=dict(width=2),
    hovertemplate="<b>Data</b>: %{x}<br><b>OEE</b>: %{y}<extra></extra>"
))

# Adicionar a linha de referência inicial (meta) - exemplo com valor 8000
fig.add_trace(go.Scatter(
    x=df_oee.index, 
    y=[80] * len(df_oee), 
    mode='lines',
    name='Meta',  # Nome da linha de meta na legenda
    line=dict(color='orange', width=2, dash='dash'),
    hoverinfo='none'
))

# Configurar o layout com fundo transparente, linhas de grade, e outras customizações
fig.update_layout(
    title={
        'text': 'DOBRA QUENTE',  # Título fixo do gráfico
        'x': 0.5,  # Centralizar o título horizontalmente
        'xanchor': 'center'
    },
    xaxis=dict(
        tickformat='%d/%m/%y',  # Formato para mostrar o ano abreviado no eixo X
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey'
    ),
    template='plotly_white',
    plot_bgcolor='rgba(0, 0, 0, 0)',  # Fundo transparente
    height=500, width=1000,
    font_color="blue",
    title_font_color="black",
    font=dict(size=15, color="Black")
)

# Criar os filtros dinâmicos para o mês
dropdown_buttons_month = [
    dict(label="Todos os meses",
         method="update",
         args=[{"x": [df_oee.index, df_oee.index], 
                "y": [df_oee['DOBRA QUENTE'], [80] * len(df_oee)]},
               {"annotations": []}]),  # Remove a atualização do título
]

# Adicionar uma opção no dropdown para cada mês
for month in months:
    filtered_df = df_oee[df_oee['Month'] == month]  # Filtra o DataFrame pelo mês atual
    dropdown_buttons_month.append(
        dict(label=month,  # Nome do mês que aparecerá no menu
             method="update",
             args=[{"x": [filtered_df.index, filtered_df.index], 
                    "y": [filtered_df['DOBRA QUENTE'], [80] * len(filtered_df)]},
                   {"annotations": []}])  # Remove a atualização do título
    )

# Adicionar os botões ao layout e posicionar na mesma linha do título
fig.update_layout(
    updatemenus=[
        dict(
            buttons=dropdown_buttons_month,
            direction="down",  # Direção do menu (dropdown)
            pad={"r": 10, "t": 10},  # Espaçamento do menu em relação ao gráfico
            showactive=True,
            x=0.05,  # Ajusta a posição horizontal do menu
            xanchor='left',
            y=1.25,  # Ajusta a posição vertical para alinhar com o título
            yanchor='top'
        )
    ]
)

# Mostrar o gráfico
fig.show()

In [ ]:
# Definir a localidade para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

# Supondo que df_oee já foi criado corretamente com as datas como índice e as unidades como colunas
df_oee.index = pd.to_datetime(df_oee.index, format='%d/%m/%Y')

# Filtrar para mostrar apenas os dados de 2024 e até o dia anterior ao atual
data_atual = datetime.now().date()
df_oee = df_oee[(df_oee.index.year == 2025) & (df_oee.index < pd.Timestamp(data_atual))]
df_oee = df_oee[~df_oee.index.weekday.isin([5, 6])]

# Extrair os meses do DataFrame em português
df_oee['Month'] = df_oee.index.strftime('%B')  # Nome do mês em português

# Filtrar os dados para um mês específico, como "Setembro"
mes = 'mar'  # Especifique o mês desejado em português (ex: 'janeiro', 'fevereiro', etc.)
filtered_df = df_oee[df_oee['Month'].str.lower() == mes.lower()]

# Certificar que os valores NaN são preenchidos para evitar quebras na linha
filtered_df = filtered_df.fillna(0)

# Caminho da imagem
image_path = "I:\\22 Gráficos e indicadores produção\\GRAMPOS\\Relatório\\AESA.png"  # Substitua pelo caminho correto da sua imagem

# Converter a imagem para base64
with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

# Criar o gráfico com subplots 2x2
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("CORTE", "APONTAMENTO", "GRAVAÇÃO", "LAMINAÇÃO"),
    shared_xaxes=False,
    shared_yaxes=False,
    vertical_spacing=0.1,
    horizontal_spacing=0.05
)

# Lista das unidades que serão plotadas
units = ['CORTE', 'APONTAMENTO', 'GRAVAÇÃO', 'LAMINAÇÃO']

# Definir a meta para cada unidade
meta_values = {
    'CORTE': 80,
    'APONTAMENTO': 80,
    'GRAVAÇÃO': 80,
    'LAMINAÇÃO': 80
}

# Adicionar cada gráfico no subplot
for i, unit in enumerate(units):
    # Determinar a posição do gráfico
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    # Calcular a média dos valores filtrados para a unidade
    mean_value = filtered_df[unit].mean()
    
    # Adicionar a linha de produção com cor azul
    show_legend_production = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=filtered_df[unit],
        mode='lines+markers',
        name='OEE',
        marker=dict(size=10),
        line=dict(color='blue', width=2),  # Cor azul para todas as linhas
        hovertemplate="<b>Data</b>: %{x}<br><b>OEE</b>: %{y}<extra></extra>",
        showlegend=show_legend_production
    ), row=row, col=col)
    
    # Adicionar a linha de meta (laranja)
    show_legend_meta = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=[meta_values[unit]] * len(filtered_df),
        mode='lines',
        name='Meta',
        line=dict(color='orange', width=2, dash='dash'),
        showlegend=show_legend_meta
    ), row=row, col=col)
    
    # Adicionar a linha da média (verde)
    show_legend_mean = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=[mean_value] * len(filtered_df),
        mode='lines',
        name='Média',
        line=dict(color='green', width=2, dash='dot'),
        showlegend=show_legend_mean
    ), row=row, col=col)

# Configurar o layout geral do gráfico
fig.update_layout(
    title={
        'text': f'OEE Mensal - {mes.capitalize()}',
        'x': 0.5,
        'xanchor': 'center'
    },
    height=1000, width=1500,
    template='plotly_white',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(size=15, color="black"),
    title_font_color="black",
    legend=dict(
        title="Legenda",
        x=1.05,  # Posicionar a legenda à direita
        y=1,     # Posicionar a legenda no topo
        traceorder="normal",
        font=dict(size=12)
    ),
    images=[dict(
        source=f"data:image/png;base64,{encoded_image}",
        xref="paper", yref="paper",
        x=0, y=1.1,
        sizex=0.12, sizey=0.12, # tamanho do logo
        xanchor="left", yanchor="top"
    )]
)

# Ajustar o formato das datas no eixo x
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(
            tickformat="%d/%m",  # Formato: dia/mês
            row=i, col=j
        )

# Mostrar o gráfico
fig.show()

In [49]:


# Definir a localidade para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

# Supondo que df_oee já foi criado corretamente com as datas como índice e as unidades como colunas
df_oee.index = pd.to_datetime(df_oee.index, format='%d/%m/%Y')

# Filtrar para mostrar apenas os dados de 2024 e até o dia anterior ao atual
data_atual = datetime.now().date()
df_oee = df_oee[(df_oee.index.year == 2025) & (df_oee.index < pd.Timestamp(data_atual))]
df_oee = df_oee[~df_oee.index.weekday.isin([5, 6])]


# Extrair os meses do DataFrame em português
df_oee['Month'] = df_oee.index.strftime('%B')  # Nome do mês em português

# Filtrar os dados para um mês específico, como "Setembro"
mes = 'março'  # Especifique o mês desejado em português (ex: 'janeiro', 'fevereiro', etc.)
filtered_df = df_oee[df_oee['Month'].str.lower() == mes.lower()]

# Certificar que os valores NaN são preenchidos para evitar quebras na linha
filtered_df = filtered_df.fillna(0)

# Caminho da imagem
image_path = "I:\\22 Gráficos e indicadores produção\\GRAMPOS\\Relatório\\AESA.png"  # Substitua pelo caminho correto da sua imagem

# Converter a imagem para base64
with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

# Criar o gráfico com subplots 2x2
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("TORNO", "TRAT. TÉRMICO", "AJUSTAGEM", "ACAB. EXTERNO"),
    shared_xaxes=False,
    shared_yaxes=False,
    vertical_spacing=0.1,
    horizontal_spacing=0.05
)

# Lista das unidades que serão plotadas
units = ['TORNO', 'TRAT. TÉRMICO', 'AJUSTAGEM', 'ACAB. EXTERNO']

# Definir a meta para cada unidade
meta_values = {
    'TORNO': 80,
    'TRAT. TÉRMICO': 80,
    'AJUSTAGEM': 80,
    'ACAB. EXTERNO': 80
}

# Adicionar cada gráfico no subplot
for i, unit in enumerate(units):
    # Determinar a posição do gráfico
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    # Calcular a média dos valores filtrados para a unidade
    mean_value = filtered_df[unit].mean()
    
    # Adicionar a linha de produção com cor azul
    show_legend_production = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=filtered_df[unit],
        mode='lines+markers',
        name='OEE',
        marker=dict(size=10),
        line=dict(color='blue', width=2),  # Cor azul para todas as linhas
        hovertemplate="<b>Data</b>: %{x}<br><b>OEE</b>: %{y}<extra></extra>",
        showlegend=show_legend_production
    ), row=row, col=col)
    
    # Adicionar a linha de meta (laranja)
    show_legend_meta = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=[meta_values[unit]] * len(filtered_df),
        mode='lines',
        name='Meta',
        line=dict(color='orange', width=2, dash='dash'),
        showlegend=show_legend_meta
    ), row=row, col=col)
    
    # Adicionar a linha da média (verde)
    show_legend_mean = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=[mean_value] * len(filtered_df),
        mode='lines',
        name='Média',
        line=dict(color='green', width=2, dash='dot'),
        showlegend=show_legend_mean
    ), row=row, col=col)

# Configurar o layout geral do gráfico
fig.update_layout(
    title={
        'text': f'OEE Mensal - {mes.capitalize()}',
        'x': 0.5,
        'xanchor': 'center'
    },
    height=1000, width=1500,
    template='plotly_white',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(size=15, color="black"),
    title_font_color="black",
    legend=dict(
        title="Legenda",
        x=1.05,  # Posicionar a legenda à direita
        y=1,     # Posicionar a legenda no topo
        traceorder="normal",
        font=dict(size=12)
    ),
    images=[dict(
        source=f"data:image/png;base64,{encoded_image}",
        xref="paper", yref="paper",
        x=0, y=1.1,
        sizex=0.12, sizey=0.12, # tamanho do logo
        xanchor="left", yanchor="top"
    )]
)

# Ajustar o formato das datas no eixo x
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(
            tickformat="%d/%m",  # Formato: dia/mês
            row=i, col=j
        )

# Mostrar o gráfico
fig.show()

In [50]:


# Definir a localidade para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

# Supondo que df_oee já foi criado corretamente com as datas como índice e as unidades como colunas
df_oee.index = pd.to_datetime(df_oee.index, format='%d/%m/%Y')

# Filtrar para mostrar apenas os dados de 2024 e até o dia anterior ao atual
data_atual = datetime.now().date()
df_oee = df_oee[(df_oee.index.year == 2025) & (df_oee.index < pd.Timestamp(data_atual))]
df_oee = df_oee[~df_oee.index.weekday.isin([5, 6])]


# Extrair os meses do DataFrame em português
df_oee['Month'] = df_oee.index.strftime('%B')  # Nome do mês em português

# Filtrar os dados para um mês específico, como "Setembro"
mes = 'março'  # Especifique o mês desejado em português (ex: 'janeiro', 'fevereiro', etc.)
filtered_df = df_oee[df_oee['Month'].str.lower() == mes.lower()]

# Certificar que os valores NaN são preenchidos para evitar quebras na linha
filtered_df = filtered_df.fillna(0)

# Caminho da imagem
image_path = "I:\\22 Gráficos e indicadores produção\\GRAMPOS\\Relatório\\AESA.png"  # Substitua pelo caminho correto da sua imagem

# Converter a imagem para base64
with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()


# Criar o gráfico com subplots 2x2
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("DOBRA FRIA", "DOBRA QUENTE", "KTL", "EMBALAGEM"),
    shared_xaxes=False,
    shared_yaxes=False,
    vertical_spacing=0.1,
    horizontal_spacing=0.05
)

# Lista das unidades que serão plotadas
units = ['DOBRA FRIA',
          'DOBRA QUENTE', 
          'KTL', 'EMBALAGEM']

# Definir a meta para cada unidade
meta_values = {
    'DOBRA FRIA': 80,
    'DOBRA QUENTE': 80,
    'KTL': 80,
    'EMBALAGEM': 80
}


# Adicionar cada gráfico no subplot
for i, unit in enumerate(units):
    # Determinar a posição do gráfico
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    # Calcular a média dos valores filtrados para a unidade
    mean_value = filtered_df[unit].mean()
    
    # Adicionar a linha de produção com cor azul
    show_legend_production = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=filtered_df[unit],
        mode='lines+markers',
        name='OEE',
        marker=dict(size=10),
        line=dict(color='blue', width=2),  # Cor azul para todas as linhas
        hovertemplate="<b>Data</b>: %{x}<br><b>OEE</b>: %{y}<extra></extra>",
        showlegend=show_legend_production
    ), row=row, col=col)
    
    # Adicionar a linha de meta (laranja)
    show_legend_meta = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=[meta_values[unit]] * len(filtered_df),
        mode='lines',
        name='Meta',
        line=dict(color='orange', width=2, dash='dash'),
        showlegend=show_legend_meta
    ), row=row, col=col)
    
    # Adicionar a linha da média (verde)
    show_legend_mean = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=[mean_value] * len(filtered_df),
        mode='lines',
        name='Média',
        line=dict(color='green', width=2, dash='dot'),
        showlegend=show_legend_mean
    ), row=row, col=col)

# Configurar o layout geral do gráfico
fig.update_layout(
    title={
        'text': f'OEE Mensal - {mes.capitalize()}',
        'x': 0.5,
        'xanchor': 'center'
    },
    height=1000, width=1500,
    template='plotly_white',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(size=15, color="black"),
    title_font_color="black",
    legend=dict(
        title="Legenda",
        x=1.05,  # Posicionar a legenda à direita
        y=1,     # Posicionar a legenda no topo
        traceorder="normal",
        font=dict(size=12)
    ),
    images=[dict(
        source=f"data:image/png;base64,{encoded_image}",
        xref="paper", yref="paper",
        x=0, y=1.1,
        sizex=0.12, sizey=0.12, # tamanho do logo
        xanchor="left", yanchor="top"
    )]
)

# Ajustar o formato das datas no eixo x
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(
            tickformat="%d/%m",  # Formato: dia/mês
            row=i, col=j
        )

# Mostrar o gráfico
fig.show()

In [51]:
# Definir a localidade para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

# Caminho onde as imagens dos gráficos serão salvas
output_dir = "I:\\22 Gráficos e indicadores produção\\GRAMPOS\\Relatório Mensal\\"

# Caminho do logo
logo_path = "I:\\22 Gráficos e indicadores produção\\GRAMPOS\\Relatório\\AESA.png"

# Filtrar para mostrar apenas os dados de 2024 e até o dia anterior ao atual
data_atual = datetime.now().date()
df_oee = df_oee[(df_oee.index.year == 2025) & (df_oee.index < pd.Timestamp(data_atual))]
df_oee = df_oee[~df_oee.index.weekday.isin([5, 6])]


# Extrair os meses do DataFrame em português
df_oee['Month'] = df_oee.index.strftime('%B')

# Filtrar os dados para o mês desejado
mes = 'março'  # Exemplo
filtered_df = df_oee[df_oee['Month'].str.lower() == mes.lower()]

# Listas das unidades por página
pages = {
    "Página 1": ['CORTE', 'APONTAMENTO', 'GRAVAÇÃO', 'LAMINAÇÃO'],
    "Página 2": ['DOBRA FRIA', 'DOBRA QUENTE', 'KTL', 'EMBALAGEM'],
    "Página 3": ['TORNO', 'TRAT. TÉRMICO', 'AJUSTAGEM', 'ACAB. EXTERNO']
}

# Definir as metas para cada unidade
meta_values = {
    'CORTE': 80, 'APONTAMENTO': 80, 'GRAVAÇÃO': 80, 'LAMINAÇÃO': 80,
    'DOBRA FRIA': 80, 'DOBRA QUENTE': 80, 'KTL': 80, 'EMBALAGEM': 80,
    'TORNO': 80, 'TRAT. TÉRMICO': 80, 'AJUSTAGEM': 80, 'ACAB. EXTERNO': 80
}

# Classe personalizada para incluir o rodapé
class PDF(FPDF):
    def footer(self):
        # Posição a 10 mm do fim
        self.set_y(-10)
        self.set_font('Arial', 'I', 8)
        # Texto do rodapé
        generation_time = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
        self.cell(0, 10, f'Depto de Engenharia - by MES AESA - {generation_time}', 0, 0, 'C')

# Criar o PDF com FPDF na orientação paisagem (L)
pdf = PDF(orientation='L', unit='mm', format='A4')
pdf.set_auto_page_break(auto=True, margin=15)

# Margens ajustadas para centralizar os gráficos
left_margin = 15
right_margin = 15
top_margin = 15

# Criar os gráficos e salvar como imagens para cada unidade e página
for page, units in pages.items():
    for unit in units:
        fig = make_subplots()
        mean_value = filtered_df[unit].mean()
        fig.add_trace(go.Scatter(
            x=filtered_df.index,
            y=filtered_df.get(unit, pd.Series([0] * len(filtered_df), index=filtered_df.index)),
            mode='lines+markers',
            name='OEE',
            marker=dict(size=8),
            line=dict(color='blue', width=2)
        ))
        fig.add_trace(go.Scatter(
            x=filtered_df.index,
            y=[meta_values[unit]] * len(filtered_df),
            mode='lines',
            name='Meta',
            line=dict(color='orange', width=2, dash='dash')
        ))
        fig.add_trace(go.Scatter(
            x=filtered_df.index,
            y=[mean_value] * len(filtered_df),
            mode='lines',
            name='Média',
            line=dict(color='green', width=2, dash='dot')
        ))
        fig.update_layout(
            title={
                'text': unit,
                'x': 0.5,
                'xanchor': 'center'
            },
            xaxis_title=None,
            yaxis_title=None,
            height=500, width=700,
             font=dict(size=12, color="black"),  # Definir a cor da fonte para preta
            template='plotly_white',
            legend=dict(
                title="Legenda",
                x=1.05,
                y=1,
                traceorder="normal"
            )
        )
        # Atualizar o formato das datas no eixo x para o formato DD/MM
        fig.update_xaxes(tickformat="%d/%m")

        # Salvar a imagem
        fig.write_image(f"{output_dir}{unit}_grafico.png")

for page, units in pages.items():
    pdf.add_page()

    # Adicionar o logo
    pdf.image(logo_path, x=left_margin, y=8, w=33)

    # Adicionar o título da página
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 5, f'OEE Mensal - {mes.capitalize()}', 0, 1, 'C')

    # Adicionar gráficos ao PDF
    pdf.set_font('Arial', '', 12)
    x_offset = left_margin
    y_offset = top_margin + 5  # Ajuste para descer o primeiro gráfico

    for i, unit in enumerate(units):
        # Determinar a posição do gráfico
        if i % 2 == 0 and i != 0:
            x_offset = left_margin
            y_offset += 90  # Aumentar o espaçamento vertical para evitar sobreposição

        # Adicionar o gráfico da unidade
        pdf.image(f"{output_dir}{unit}_grafico.png", x=x_offset, y=y_offset, w=130)
        x_offset += 130  # Mover para a próxima coluna com espaçamento ajustado

# Salvar o PDF final
pdf.output(f"{output_dir}OEE_MENSAL.pdf")
print(f"PDF salvo em: {output_dir}OEE_MENSAL.pdf")
 

PDF salvo em: I:\22 Gráficos e indicadores produção\GRAMPOS\Relatório Mensal\OEE_MENSAL.pdf


In [60]:
df_pivot.index = pd.to_datetime(df_pivot.index, format='%d/%m/%Y')

# Filtrar para mostrar apenas os dados de 2024
df_pivot = df_pivot[df_pivot.index.year == 2025]

# Filtrar para mostrar apenas os dados de 2024 e até o dia anterior ao atual
data_atual = datetime.now().date()
df_pivot = df_pivot[(df_pivot.index.year == 2025) & (df_pivot.index < pd.Timestamp(data_atual))]
df_pivot = df_pivot[~df_pivot.index.weekday.isin([5, 6])]


# Extrair os meses do DataFrame para criar os filtros
df_pivot['Month'] = df_pivot.index.strftime('%B')  # Adiciona uma coluna com o nome do mês
months = df_pivot['Month'].unique()

# Criar o gráfico de linha com marcadores para a coluna 'CORTE'
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.05)

# Adicionar a linha com os pontos de produção para 'CORTE' (inicialmente mostrando todos os meses de 2024)
fig.add_trace(go.Scatter(
    x=df_pivot.index, 
    y=df_pivot['DOBRA QUENTE'],
    mode='lines+markers',
    name='Produção',  # Nome da linha na legenda
    marker=dict(size=8),
    line=dict(width=2),
    hovertemplate="<b>Data</b>: %{x}<br><b>Produção</b>: %{y}<extra></extra>"
))

# Adicionar a linha de referência inicial (meta) - exemplo com valor 8000
fig.add_trace(go.Scatter(
    x=df_pivot.index, 
    y=[8000] * len(df_pivot), 
    mode='lines',
    name='Meta',  # Nome da linha de meta na legenda
    line=dict(color='orange', width=2, dash='dash'),
    hoverinfo='none'
))

# Configurar o layout com fundo transparente, linhas de grade, e outras customizações
fig.update_layout(
    title={
        'text': 'DOBRA QUENTE',  # Título fixo do gráfico
        'x': 0.5,  # Centralizar o título horizontalmente
        'xanchor': 'center'
    },
    xaxis=dict(
        tickformat='%d/%m/%y',  # Formato para mostrar o ano abreviado no eixo X
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey'
    ),
    template='plotly_white',
    plot_bgcolor='rgba(0, 0, 0, 0)',  # Fundo transparente
    height=500, width=1000,
    font_color="blue",
    title_font_color="black",
    font=dict(size=15, color="Black")
)

# Criar os filtros dinâmicos para o mês
dropdown_buttons_month = [
    dict(label="Todos os meses",
         method="update",
         args=[{"x": [df_pivot.index, df_pivot.index], 
                "y": [df_pivot['DOBRA QUENTE'], [8000] * len(df_pivot)]},
               {"annotations": []}]),  # Remove a atualização do título
]

# Adicionar uma opção no dropdown para cada mês
for month in months:
    filtered_df = df_pivot[df_pivot['Month'] == month]  # Filtra o DataFrame pelo mês atual
    dropdown_buttons_month.append(
        dict(label=month,  # Nome do mês que aparecerá no menu
             method="update",
             args=[{"x": [filtered_df.index, filtered_df.index], 
                    "y": [filtered_df['CORTE'], [8000] * len(filtered_df)]},
                   {"annotations": []}])  # Remove a atualização do título
    )

# Adicionar os botões ao layout e posicionar na mesma linha do título
fig.update_layout(
    updatemenus=[
        dict(
            buttons=dropdown_buttons_month,
            direction="down",  # Direção do menu (dropdown)
            pad={"r": 10, "t": 10},  # Espaçamento do menu em relação ao gráfico
            showactive=True,
            x=0.25,  # Ajusta a posição horizontal do menu
            xanchor='left',
            y=1.25,  # Ajusta a posição vertical para alinhar com o título
            yanchor='top'
        )
    ]
)

# Mostrar o gráfico
fig.show()

In [53]:
# Definir a localidade para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

# Supondo que df_pivot já foi criado corretamente com as datas como índice e as unidades como colunas
df_pivot.index = pd.to_datetime(df_pivot.index, format='%d/%m/%Y')

# Filtrar para mostrar apenas os dados de 2024 e até o dia anterior ao atual
data_atual = datetime.now().date()
df_pivot = df_pivot[(df_pivot.index.year == 2025) & (df_pivot.index < pd.Timestamp(data_atual))]
df_pivot = df_pivot[~df_pivot.index.weekday.isin([5, 6])]



# Extrair os meses do DataFrame em português
df_pivot['Month'] = df_pivot.index.strftime('%B')  # Nome do mês em português

# Filtrar os dados para um mês específico, como "Setembro"
mes = 'março'  # Especifique o mês desejado em português (ex: 'janeiro', 'fevereiro', etc.)
filtered_df = df_pivot[df_pivot['Month'].str.lower() == mes.lower()]

# Certificar que os valores NaN são preenchidos para evitar quebras na linha
filtered_df = filtered_df.fillna(0)

# Caminho da imagem
image_path = "I:\\22 Gráficos e indicadores produção\\GRAMPOS\\Relatório\\AESA.png"  # Substitua pelo caminho correto da sua imagem

# Converter a imagem para base64
with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

# Criar o gráfico com subplots 2x2
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("CORTE", "APONTAMENTO", "GRAVAÇÃO", "LAMINAÇÃO"),
    shared_xaxes=False,
    shared_yaxes=False,
    vertical_spacing=0.1,
    horizontal_spacing=0.05
)

# Lista das unidades que serão plotadas
units = ['CORTE', 'APONTAMENTO', 'GRAVAÇÃO', 'LAMINAÇÃO']

# Definir a meta para cada unidade
meta_values = {
    'CORTE': 8000,
    'APONTAMENTO': 5000,
    'GRAVAÇÃO': 4000,
    'LAMINAÇÃO': 6000
}

# Adicionar cada gráfico no subplot
for i, unit in enumerate(units):
    # Determinar a posição do gráfico
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    # Calcular a média dos valores filtrados para a unidade
    mean_value = filtered_df[unit].mean()
    
    # Adicionar a linha de produção com cor azul
    show_legend_production = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=filtered_df[unit],
        mode='lines+markers',
        name='Produção',
        marker=dict(size=10),
        line=dict(color='blue', width=2),  # Cor azul para todas as linhas
        hovertemplate="<b>Data</b>: %{x}<br><b>Produção</b>: %{y}<extra></extra>",
        showlegend=show_legend_production
    ), row=row, col=col)
    
    # Adicionar a linha de meta (laranja)
    show_legend_meta = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=[meta_values[unit]] * len(filtered_df),
        mode='lines',
        name='Meta',
        line=dict(color='orange', width=2, dash='dash'),
        showlegend=show_legend_meta
    ), row=row, col=col)
    
    # Adicionar a linha da média (verde)
    show_legend_mean = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=[mean_value] * len(filtered_df),
        mode='lines',
        name='Média',
        line=dict(color='green', width=2, dash='dot'),
        showlegend=show_legend_mean
    ), row=row, col=col)

# Configurar o layout geral do gráfico
fig.update_layout(
    title={
        'text': f'Relatório Mensal - {mes.capitalize()}',
        'x': 0.5,
        'xanchor': 'center'
    },
    height=1000, width=1500,
    template='plotly_white',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(size=15, color="black"),
    title_font_color="black",
    legend=dict(
        title="Legenda",
        x=1.05,  # Posicionar a legenda à direita
        y=1,     # Posicionar a legenda no topo
        traceorder="normal",
        font=dict(size=12)
    ),
    images=[dict(
        source=f"data:image/png;base64,{encoded_image}",
        xref="paper", yref="paper",
        x=0, y=1.1,
        sizex=0.12, sizey=0.12, # tamanho do logo
        xanchor="left", yanchor="top"
    )]
)

# Ajustar o formato das datas no eixo x
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(
            tickformat="%d/%m",  # Formato: dia/mês
            row=i, col=j
        )

# Mostrar o gráfico
fig.show()


In [54]:
# Definir a localidade para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

# Supondo que df_pivot já foi criado corretamente com as datas como índice e as unidades como colunas
df_pivot.index = pd.to_datetime(df_pivot.index, format='%d/%m/%Y')

# Filtrar para mostrar apenas os dados de 2024 e até o dia anterior ao atual
data_atual = datetime.now().date()
df_pivot = df_pivot[(df_pivot.index.year == 2025) & (df_pivot.index < pd.Timestamp(data_atual))]
df_pivot = df_pivot[~df_pivot.index.weekday.isin([5, 6])]

# Extrair os meses do DataFrame em português
df_pivot['Month'] = df_pivot.index.strftime('%B')  # Nome do mês em português

# Filtrar os dados para um mês específico, como "Setembro"
mes = 'março'  # Especifique o mês desejado em português (ex: 'janeiro', 'fevereiro', etc.)
filtered_df = df_pivot[df_pivot['Month'].str.lower() == mes.lower()]

# Certificar que os valores NaN são preenchidos para evitar quebras na linha
filtered_df = filtered_df.fillna(0)

# Caminho da imagem
image_path = "I:\\22 Gráficos e indicadores produção\\GRAMPOS\\Relatório\\AESA.png"  # Substitua pelo caminho correto da sua imagem

# Converter a imagem para base64
with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

# Criar o gráfico com subplots 2x2
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("DOBRA FRIA", "DOBRA QUENTE", "KTL", "EMBALAGEM"),
    shared_xaxes=False,
    shared_yaxes=False,
    vertical_spacing=0.1,
    horizontal_spacing=0.05
)

# Lista das unidades que serão plotadas
units = ['DOBRA FRIA', 'DOBRA QUENTE', 'KTL', 'EMBALAGEM']

# Definir a meta para cada unidade
meta_values = {
    'DOBRA FRIA': 6000,
    'DOBRA QUENTE': 3500,
    'KTL': 5000,
    'EMBALAGEM': 5000
}

# Adicionar cada gráfico no subplot
for i, unit in enumerate(units):
    # Determinar a posição do gráfico
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    # Calcular a média dos valores filtrados para a unidade
    mean_value = filtered_df[unit].mean()
    
    # Adicionar a linha de produção com cor azul
    show_legend_production = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=filtered_df[unit],
        mode='lines+markers',
        name='Produção',
        marker=dict(size=10),
        line=dict(color='blue', width=2),  # Cor azul para todas as linhas
        hovertemplate="<b>Data</b>: %{x}<br><b>Produção</b>: %{y}<extra></extra>",
        showlegend=show_legend_production
    ), row=row, col=col)
    
    # Adicionar a linha de meta (laranja)
    show_legend_meta = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=[meta_values[unit]] * len(filtered_df),
        mode='lines',
        name='Meta',
        line=dict(color='orange', width=2, dash='dash'),
        showlegend=show_legend_meta
    ), row=row, col=col)
    
    # Adicionar a linha da média (verde)
    show_legend_mean = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=[mean_value] * len(filtered_df),
        mode='lines',
        name='Média',
        line=dict(color='green', width=2, dash='dot'),
        showlegend=show_legend_mean
    ), row=row, col=col)

# Configurar o layout geral do gráfico
fig.update_layout(
    title={
        'text': f'Relatório Mensal - {mes.capitalize()}',
        'x': 0.5,
        'xanchor': 'center'
    },
    height=1000, width=1500,
    template='plotly_white',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(size=15, color="black"),
    title_font_color="black",
    legend=dict(
        title="Legenda",
        x=1.05,  # Posicionar a legenda à direita
        y=1,     # Posicionar a legenda no topo
        traceorder="normal",
        font=dict(size=12)
    ),
    images=[dict(
        source=f"data:image/png;base64,{encoded_image}",
        xref="paper", yref="paper",
        x=0, y=1.1,
        sizex=0.12, sizey=0.12, # tamanho do logo
        xanchor="left", yanchor="top"
    )]
)

# Ajustar o formato das datas no eixo x
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(
            tickformat="%d/%m",  # Formato: dia/mês
            row=i, col=j
        )

# Mostrar o gráfico
fig.show()


In [55]:
# Definir a localidade para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

# Supondo que df_pivot já foi criado corretamente com as datas como índice e as unidades como colunas
df_pivot.index = pd.to_datetime(df_pivot.index, format='%d/%m/%Y')
df_pivot = df_pivot[~df_pivot.index.weekday.isin([5, 6])]


# Extrair os meses do DataFrame em português
df_pivot['Month'] = df_pivot.index.strftime('%B')  # Nome do mês em português

# Filtrar os dados para um mês específico, como "Setembro"
mes = 'março'  # Especifique o mês desejado em português (ex: 'janeiro', 'fevereiro', etc.)
filtered_df = df_pivot[df_pivot['Month'].str.lower() == mes.lower()]

# Certificar que os valores NaN são preenchidos para evitar quebras na linha
filtered_df = filtered_df.fillna(0)

# Caminho da imagem
image_path = "I:\\22 Gráficos e indicadores produção\\GRAMPOS\\Relatório\\AESA.png"  # Substitua pelo caminho correto da sua imagem

# Converter a imagem para base64
with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

# Criar o gráfico com subplots 2x2
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("TORNO", "TRAT. TÉRMICO", "AJUSTAGEM", "ACAB. EXTERNO"),
    shared_xaxes=False,
    shared_yaxes=False,
    vertical_spacing=0.1,
    horizontal_spacing=0.05
)

# Lista das unidades que serão plotadas
units = ['TORNO', 'TRAT. TÉRMICO', 'AJUSTAGEM', 'ACAB. EXTERNO']

# Definir a meta para cada unidade
meta_values = {
    'TORNO': 900,
    'TRAT. TÉRMICO': 3000,
    'AJUSTAGEM': 4000,
    'ACAB. EXTERNO': 4000
}

# Adicionar cada gráfico no subplot
for i, unit in enumerate(units):
    # Determinar a posição do gráfico
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    # Calcular a média dos valores filtrados para a unidade
    mean_value = filtered_df[unit].mean()
    
    # Adicionar a linha de produção com cor azul
    show_legend_production = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=filtered_df[unit],
        mode='lines+markers',
        name='Produção',
        marker=dict(size=10),
        line=dict(color='blue', width=2),  # Cor azul para todas as linhas
        hovertemplate="<b>Data</b>: %{x}<br><b>Produção</b>: %{y}<extra></extra>",
        showlegend=show_legend_production
    ), row=row, col=col)
    
    # Adicionar a linha de meta (laranja)
    show_legend_meta = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=[meta_values[unit]] * len(filtered_df),
        mode='lines',
        name='Meta',
        line=dict(color='orange', width=2, dash='dash'),
        showlegend=show_legend_meta
    ), row=row, col=col)
    
    # Adicionar a linha da média (verde)
    show_legend_mean = i == 0  # Mostrar legenda apenas para o primeiro gráfico
    fig.add_trace(go.Scatter(
        x=filtered_df.index,
        y=[mean_value] * len(filtered_df),
        mode='lines',
        name='Média',
        line=dict(color='green', width=2, dash='dot'),
        showlegend=show_legend_mean
    ), row=row, col=col)

# Configurar o layout geral do gráfico
fig.update_layout(
    title={
        'text': f'Relatório Mensal - {mes.capitalize()}',
        'x': 0.5,
        'xanchor': 'center'
    },
    height=1000, width=1500,
    template='plotly_white',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(size=15, color="black"),
    title_font_color="black",
    legend=dict(
        title="Legenda",
        x=1.05,  # Posicionar a legenda à direita
        y=1,     # Posicionar a legenda no topo
        traceorder="normal",
        font=dict(size=12)
    ),
    images=[dict(
        source=f"data:image/png;base64,{encoded_image}",
        xref="paper", yref="paper",
        x=0, y=1.1,
        sizex=0.12, sizey=0.12, # tamanho do logo
        xanchor="left", yanchor="top"
    )]
)

# Ajustar o formato das datas no eixo x
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(
            tickformat="%d/%m",  # Formato: dia/mês
            row=i, col=j
        )

# Mostrar o gráfico
fig.show()

In [56]:
# Definir a localidade para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

# Caminho onde as imagens dos gráficos serão salvas
output_dir = "I:\\22 Gráficos e indicadores produção\\GRAMPOS\\Relatório Mensal\\"

# Caminho do logo
logo_path = "I:\\22 Gráficos e indicadores produção\\GRAMPOS\\Relatório\\AESA.png"

# Extrair os meses do DataFrame em português
df_pivot['Month'] = df_pivot.index.strftime('%B')

# Filtrar para mostrar apenas os dados de 2024 e até o dia anterior ao atual
data_atual = datetime.now().date()
df_pivot = df_pivot[(df_pivot.index.year == 2025) & (df_pivot.index < pd.Timestamp(data_atual))]
df_pivot = df_pivot[~df_pivot.index.weekday.isin([5, 6])]


# Filtrar os dados para o mês desejado
mes = 'março'  # Exemplo
filtered_df = df_pivot[df_pivot['Month'].str.lower() == mes.lower()]

# Listas das unidades por página
pages = {
    "Página 1": ['CORTE', 'APONTAMENTO', 'GRAVAÇÃO', 'LAMINAÇÃO'],
    "Página 2": ['DOBRA FRIA', 'DOBRA QUENTE', 'KTL', 'EMBALAGEM'],
    "Página 3": ['TORNO', 'TRAT. TÉRMICO', 'AJUSTAGEM', 'ACAB. EXTERNO']
}

# Definir as metas para cada unidade
meta_values = {
    'CORTE': 80, 'APONTAMENTO': 80, 'GRAVAÇÃO': 80, 'LAMINAÇÃO': 80,
    'DOBRA FRIA': 80, 'DOBRA QUENTE': 80, 'KTL': 80, 'EMBALAGEM': 80,
    'TORNO': 80, 'TRAT. TÉRMICO': 80, 'AJUSTAGEM': 80, 'ACAB. EXTERNO': 80
}

# Criar os gráficos e salvar como imagens para cada unidade e página
for page, units in pages.items():
    for unit in units:
        fig = make_subplots()
        mean_value = filtered_df[unit].mean()
        fig.add_trace(go.Scatter(
            x=filtered_df.index,
            y=filtered_df.get(unit, pd.Series([0] * len(filtered_df), index=filtered_df.index)),
            mode='lines+markers',
            name='Produção',
            marker=dict(size=8),
            line=dict(color='blue', width=2)
        ))
        fig.add_trace(go.Scatter(
            x=filtered_df.index,
            y=[meta_values[unit]] * len(filtered_df),
            mode='lines',
            name='Meta',
            line=dict(color='orange', width=2, dash='dash')
        ))
        fig.add_trace(go.Scatter(
            x=filtered_df.index,
            y=[mean_value] * len(filtered_df),
            mode='lines',
            name='Média',
            line=dict(color='green', width=2, dash='dot')
        ))
        fig.update_layout(
            title=f'{unit} - {mes.capitalize()}',
            height=400, width=600,
            template='plotly_white',
            legend=dict(
                title="Legenda",
                x=1.05,
                y=1,
                traceorder="normal"
            )
        )
        # Salvar a imagem
        fig.write_image(f"{output_dir}{unit}_grafico.png")

# Criar o PDF com FPDF na orientação paisagem (L)
pdf = FPDF(orientation='L', unit='mm', format='A4')
pdf.set_auto_page_break(auto=True, margin=15)

for page, units in pages.items():
    pdf.add_page()

    # Adicionar o logo
    pdf.image(logo_path, 10, 8, 33)

    # Adicionar o título da página
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, f'Relatório Mensal - {mes.capitalize()}', 0, 1, 'C')

    # Adicionar gráficos ao PDF
    pdf.set_font('Arial', '', 12)
    x_offset = 10
    y_offset = 40
    for i, unit in enumerate(units):
        # Determinar a posição do gráfico
        if i % 2 == 0 and i != 0:
            x_offset = 10
            y_offset += 100  # Pular linha após dois gráficos

        # Adicionar o gráfico da unidade
        pdf.image(f"{output_dir}{unit}_grafico.png", x=x_offset, y=y_offset, w=130)  # Ajuste de largura para orientação paisagem
        x_offset += 150  # Mover para a próxima coluna

# Salvar o PDF final
pdf.output(f"{output_dir}Relatorio_Mensal_Completo_Paisagem.pdf")


''

In [57]:
# Definir a localidade para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

# Caminho onde as imagens dos gráficos serão salvas
output_dir = "I:\\22 Gráficos e indicadores produção\\GRAMPOS\\Relatório Mensal\\"

# Caminho do logo
logo_path = "I:\\22 Gráficos e indicadores produção\\GRAMPOS\\Relatório\\AESA.png"

# Filtrar para mostrar apenas os dados de 2024 e até o dia anterior ao atual
data_atual = datetime.now().date()
df_pivot = df_pivot[(df_pivot.index.year == 2025) & (df_pivot.index < pd.Timestamp(data_atual))]
df_pivot = df_pivot[~df_pivot.index.weekday.isin([5, 6])]


# Extrair os meses do DataFrame em português
df_pivot['Month'] = df_pivot.index.strftime('%B')

# Filtrar os dados para o mês desejado
mes = 'março'  # Exemplo
filtered_df = df_pivot[df_pivot['Month'].str.lower() == mes.lower()]

# Verificar se há dados ausentes ou nulos
filtered_df = filtered_df.fillna(0)  # Substituir valores nulos por 0

# Listas das unidades por página
pages = {
    "Página 1": ['CORTE', 'APONTAMENTO', 'GRAVAÇÃO', 'LAMINAÇÃO'],
    "Página 2": ['DOBRA FRIA', 'DOBRA QUENTE', 'KTL', 'EMBALAGEM'],
    "Página 3": ['TORNO', 'TRAT. TÉRMICO', 'AJUSTAGEM', 'ACAB. EXTERNO']
}

# Definir as metas para cada unidade
meta_values = {
    'CORTE': 8000, 'APONTAMENTO': 5000, 'GRAVAÇÃO': 4000, 'LAMINAÇÃO': 6000,
    'DOBRA FRIA': 6000, 'DOBRA QUENTE': 3500, 'KTL': 5000, 'EMBALAGEM': 5000,
    'TORNO': 900, 'TRAT. TÉRMICO': 3000, 'AJUSTAGEM': 4000, 'ACAB. EXTERNO': 4000
}

# Classe personalizada para incluir o rodapé
class PDF(FPDF):
    def footer(self):
        # Posição a 10 mm do fim
        self.set_y(-10)
        self.set_font('Arial', 'I', 8)
        # Texto do rodapé
        generation_time = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
        self.cell(0, 10, f'Depto de Engenharia - by MES AESA - {generation_time}', 0, 0, 'C')

# Criar o PDF com FPDF na orientação paisagem (L)
pdf = PDF(orientation='L', unit='mm', format='A4')
pdf.set_auto_page_break(auto=True, margin=15)

# Margens ajustadas para centralizar os gráficos
left_margin = 15
right_margin = 15
top_margin = 15

# Criar os gráficos e salvar como imagens para cada unidade e página
for page, units in pages.items():
    for unit in units:
        fig = make_subplots()
        mean_value = filtered_df[unit].mean()
        fig.add_trace(go.Scatter(
            x=filtered_df.index,
            y=filtered_df.get(unit, pd.Series([0] * len(filtered_df), index=filtered_df.index)),
            mode='lines+markers',
            name='Produção',
            marker=dict(size=8),
            line=dict(color='blue', width=2)
        ))
        fig.add_trace(go.Scatter(
            x=filtered_df.index,
            y=[meta_values[unit]] * len(filtered_df),
            mode='lines',
            name='Meta',
            line=dict(color='orange', width=2, dash='dash')
        ))
        fig.add_trace(go.Scatter(
            x=filtered_df.index,
            y=[mean_value] * len(filtered_df),
            mode='lines',
            name='Média',
            line=dict(color='green', width=2, dash='dot')
        ))
        fig.update_layout(
            title={
                'text': unit,
                'x': 0.5,
                'xanchor': 'center'
            },
            xaxis_title=None,
            yaxis_title=None,
            height=500, width=700,
             font=dict(size=12, color="black"),  # Definir a cor da fonte para preta
            template='plotly_white',
            legend=dict(
                title="Legenda",
                x=1.05,
                y=1,
                traceorder="normal"
            )
        )
        # Atualizar o formato das datas no eixo x para o formato DD/MM
        fig.update_xaxes(tickformat="%d/%m")

        # Salvar a imagem
        fig.write_image(f"{output_dir}{unit}_grafico.png")

for page, units in pages.items():
    pdf.add_page()

    # Adicionar o logo
    pdf.image(logo_path, x=left_margin, y=8, w=33)

    # Adicionar o título da página
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 5, f'Relatório Mensal - {mes.capitalize()}', 0, 1, 'C')

    # Adicionar gráficos ao PDF
    pdf.set_font('Arial', '', 12)
    x_offset = left_margin
    y_offset = top_margin + 5  # Ajuste para descer o primeiro gráfico

    for i, unit in enumerate(units):
        # Determinar a posição do gráfico
        if i % 2 == 0 and i != 0:
            x_offset = left_margin
            y_offset += 90  # Aumentar o espaçamento vertical para evitar sobreposição

        # Adicionar o gráfico da unidade
        pdf.image(f"{output_dir}{unit}_grafico.png", x=x_offset, y=y_offset, w=130)
        x_offset += 130  # Mover para a próxima coluna com espaçamento ajustado

# Salvar o PDF final
pdf.output(f"{output_dir}Relatorio_Mensal.pdf")
print(f"PDF salvo em: {output_dir}Relatorio_Mensal.pdf")

PDF salvo em: I:\22 Gráficos e indicadores produção\GRAMPOS\Relatório Mensal\Relatorio_Mensal.pdf
